# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice=T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    atoms     = [ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions, [:lda_x, :lda_c_vwn];
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770875123350                   -0.52    9.0         
  2   -2.772147049193       -2.90       -1.32    1.0    140ms
  3   -2.772170373174       -4.63       -2.46    1.0    164ms
  4   -2.772170649964       -6.56       -3.17    1.0    144ms
  5   -2.772170721728       -7.14       -3.94    2.0    187ms
  6   -2.772170722694       -9.02       -4.33    1.0    149ms
  7   -2.772170723010       -9.50       -5.22    1.0    163ms
  8   -2.772170723014      -11.43       -5.63    1.0    156ms
  9   -2.772170723015      -12.07       -7.14    2.0    188ms
 10   -2.772170723015      -14.12       -6.92    2.0    204ms
 11   -2.772170723015      -14.45       -7.37    1.0    158ms
 12   -2.772170723015   +  -14.07       -8.19    1.0    178ms
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.7

1.7735578059919654

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770681656951                   -0.53    8.0         
  2   -2.772049943664       -2.86       -1.31    1.0    140ms
  3   -2.772082631106       -4.49       -2.58    1.0    156ms
  4   -2.772083410518       -6.11       -3.74    2.0    215ms
  5   -2.772083416918       -8.19       -4.05    2.0    166ms
  6   -2.772083417788       -9.06       -5.14    1.0    178ms
  7   -2.772083417810      -10.65       -5.88    2.0    170ms
  8   -2.772083417811      -12.78       -6.52    1.0    154ms
  9   -2.772083417811      -13.48       -7.22    2.0    199ms
 10   -2.772083417811      -14.45       -7.95    1.0    165ms
 11   -2.772083417811   +  -14.45       -8.74    2.0    192ms

Polarizability via ForwardDiff:       1.772534971436127
Polarizability via finite difference: 1.7735578059919654
